# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

## Chain Class 基类

类继承关系：

```
Chain --> <name>Chain  # Examples: LLMChain, MapReduceChain, RouterChain
```

**代码实现：https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/chains/base.py**

```python
# 定义一个名为Chain的基础类
class Chain(Serializable, Runnable[Dict[str, Any], Dict[str, Any]], ABC):
    """为创建结构化的组件调用序列的抽象基类。
    
    链应该用来编码对组件的一系列调用，如模型、文档检索器、其他链等，并为此序列提供一个简单的接口。
    
    Chain接口使创建应用程序变得容易，这些应用程序是：
    - 有状态的：给任何Chain添加Memory可以使它具有状态，
    - 可观察的：向Chain传递Callbacks来执行额外的功能，如记录，这在主要的组件调用序列之外，
    - 可组合的：Chain API足够灵活，可以轻松地将Chains与其他组件结合起来，包括其他Chains。
    
    链公开的主要方法是：
    - `__call__`：链是可以调用的。`__call__`方法是执行Chain的主要方式。它将输入作为一个字典接收，并返回一个字典输出。
    - `run`：一个方便的方法，它以args/kwargs的形式接收输入，并将输出作为字符串或对象返回。这种方法只能用于一部分链，不能像`__call__`那样返回丰富的输出。
    """

    # 调用链
    def invoke(
        self, input: Dict[str, Any], config: Optional[runnableConfig] = None
    ) -> Dict[str, Any]:
        """传统调用方法。"""
        return self(input, **(config or {}))

    # 链的记忆，保存状态和变量
    memory: Optional[BaseMemory] = None
    """可选的内存对象，默认为None。
    内存是一个在每个链的开始和结束时被调用的类。在开始时，内存加载变量并在链中传递它们。在结束时，它保存任何返回的变量。
    有许多不同类型的内存，请查看内存文档以获取完整的目录。"""

    # 回调，可能用于链的某些操作或事件。
    callbacks: Callbacks = Field(default=None, exclude=True)
    """可选的回调处理程序列表（或回调管理器）。默认为None。
    在对链的调用的生命周期中，从on_chain_start开始，到on_chain_end或on_chain_error结束，都会调用回调处理程序。
    每个自定义链可以选择调用额外的回调方法，详细信息请参见Callback文档。"""

    # 是否详细输出模式
    verbose: bool = Field(default_factory=_get_verbosity)
    """是否以详细模式运行。在详细模式下，一些中间日志将打印到控制台。默认值为`langchain.verbose`。"""

    # 与链关联的标签
    tags: Optional[List[str]] = None
    """与链关联的可选标签列表，默认为None。
    这些标签将与对这个链的每次调用关联起来，并作为参数传递给在`callbacks`中定义的处理程序。
    你可以使用这些来例如识别链的特定实例与其用例。"""

    # 与链关联的元数据
    metadata: Optional[Dict[str, Any]] = None
    """与链关联的可选元数据，默认为None。
    这些元数据将与对这个链的每次调用关联起来，并作为参数传递给在`callbacks`中定义的处理程序。
    你可以使用这些来例如识别链的特定实例与其用例。"""
```

## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

In [1]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0.9, max_tokens=500)

In [2]:
prompt = PromptTemplate(
    input_variables=["product"],
    template="给制造{product}的有限公司取10个好名字，并给出完整的公司名称",
)

In [3]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)
print(chain.run({
    'product': "性能卓越的GPU"
    }))



1. 电脑科技公司（Computer Technology Corporation）
2. 星空GPU（Starry GPU）
3. 超级处理器公司（Super Processor Corporation）
4. 快速处理有限公司（Quick Processing Limited）
5. GPU芯片梦想公司（GPU Chip Dream Corporation）
6. 科技驱动特性公司（Technology Driven Features Corporation）
7. 神技超级GPU（Miracle Super GPU）
8. 显示中心GPU公司（Display Center GPU Corporation）
9. 显卡精选公司（Graphics Card Selection Corporation）
10. 快速GPU技术有限公司（Fast GPU Technologies Limited）


In [4]:
chain.verbose = True

In [5]:
chain.verbose

True

In [6]:
print(chain.run({
    'product': "性能卓越的GPU"
    }))



> Entering new LLMChain chain...
Prompt after formatting:
给制造性能卓越的GPU的有限公司取10个好名字，并给出完整的公司名称

> Finished chain.
：

1、性能奇观有限公司（Performance Wondrous Ltd.）
2、芯片卓越有限公司（Chip Excellency Ltd.）
3、GPU头脑有限公司（GPU Brainpower Ltd.）
4、均衡尖端有限公司（Balanced State-of-the-Art Ltd.）
5、芯片尖端有限公司（Chip Cutting Edge Ltd.）
6、GPU精英有限公司（GPU Elites Ltd.）
7、高性能创新有限公司（High Performance Innovation Ltd.）
8、最佳芯片有限公司（Top-Notch Chip Ltd.）
9、GPU旗舰有限公司（GPU Flagship Ltd.）
10、超级制造有限公司（Super Manufacturing Ltd.）


## Sequential Chain

串联式调用语言模型（将一个调用的输出作为另一个调用的输入）。

顺序链（Sequential Chain ）允许用户连接多个链并将它们组合成执行特定场景的流水线（Pipeline）。有两种类型的顺序链：

- SimpleSequentialChain：最简单形式的顺序链，每个步骤都具有单一输入/输出，并且一个步骤的输出是下一个步骤的输入。
- SequentialChain：更通用形式的顺序链，允许多个输入/输出。

### 使用 SimpleSequentialChain 实现戏剧摘要和评论（单输入/单输出）

![](../images/simple_sequential_chain_0.png)

In [7]:
# 这是一个 LLMChain，用于根据剧目的标题撰写简介。

llm = OpenAI(temperature=0.7, max_tokens=1000)

template = """你是一位剧作家。根据戏剧的标题，你的任务是为该标题写一个简介。

标题：{title}
剧作家：以下是对上述戏剧的简介："""

prompt_template = PromptTemplate(input_variables=["title"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template)

In [8]:
# 这是一个LLMChain，用于根据剧情简介撰写一篇戏剧评论。
# llm = OpenAI(temperature=0.7, max_tokens=1000)
template = """你是《纽约时报》的戏剧评论家。根据剧情简介，你的工作是为该剧撰写一篇评论。

剧情简介：
{synopsis}

以下是来自《纽约时报》戏剧评论家对上述剧目的评论："""

prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template)

![](../images/simple_sequential_chain_1.png)

In [9]:
# 这是一个SimpleSequentialChain，按顺序运行这两个链
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(chains=[synopsis_chain, review_chain], verbose=True)

In [10]:
review = overall_chain.run("三体人不是无法战胜的")



> Entering new SimpleSequentialChain chain...


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIError: The server had an error processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (Please include the request ID bec0bbbffa77031ba13b511f377c30cd in your email.) {
  "error": {
    "message": "The server had an error processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (Please include the request ID bec0bbbffa77031ba13b511f377c30cd in your email.)",
    "type": "server_error",
    "param": null,
    "code": null
  }
}
 500 {'error': {'message': 'The server had an error processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (Please include the request ID bec0bb



《三体人不是无法战胜的》讲述一个发生在宇宙中的有趣的故事：一位年轻的宇航员，他发现自己被困在一个叫做“三体人”的可怕种族中。他开始思考着如何摆脱这种恐怖，但最终发现他不仅能够摆脱，而且能够战胜他们。他计划着怎样才能赢得最终胜利，并最终取得成功。本剧将带领我们走进一段充满欢乐和希望的旅程，让我们在舞台上看到勇气、希望和胜利的光芒。


《三体人不是无法战胜的》是一部宏大而又充满激情的舞台剧，描绘了一个虚构的宇宙中，一位勇敢的宇航员如何战胜可怕的“三体人”种族的故事。本剧以极具活力和激情的舞台表演，勾勒出一幕幕令人惊叹的戏剧场景，让观众在充满悬念的故事中，拥抱希望与自信，共同体验一段充满欢乐和希望的旅程。该剧的演员表演精湛，把观众带到一个充满梦想与热情的世界，让我们相信，无论面对怎样的挑战，勇敢面对它们，总有一天我们会取得胜利。《三体人不是无法战胜的》是一部令人鼓舞的舞台剧，值得一看。

> Finished chain.


In [11]:
review = overall_chain.run("星球大战第九季")



> Entering new SimpleSequentialChain chain...


《星球大战第九季》是一部经典的科幻戏剧，讲述了一支太空船上的英雄们与他们古老的敌人的冒险之旅。船上的乘客们在星球之间旅行，展开一段惊心动魄的冒险，以及一段激动人心的斗争，以拯救他们从古老的敌人手中拯救宇宙。在这场大战中，敌人会用一切可能的手段来阻止他们，而英雄们也会不惜一切代价来保护宇宙的和平。这个故事将带给观众一段精彩绝伦的冒险之旅，让他们体验到一种既激动又令人振奋的情绪。


《星球大战第九季》带给观众的是一段激动人心的冒险之旅，令人振奋的情绪更是让人难以忘怀。该剧描述了一支太空船上的英雄们与古老敌人的斗争，他们不惜一切代价来保护宇宙的和平。剧中的情节悬念连连，紧张刺激，让观众沉浸其中，深深体会到英雄们的决心和坚持。在这场大战中，特效精美，动作紧凑，让观众置身其中。总之，《星球大战第九季》无疑是一部精彩绝伦的科幻戏剧，让观众体验到一种既激动又令人振奋的情绪。

> Finished chain.


### 使用 SequentialChain 实现戏剧摘要和评论（多输入/多输出）

![](../images/sequential_chain_0.png)

In [16]:
# # 这是一个 LLMChain，根据剧名和设定的时代来撰写剧情简介。
llm = OpenAI(temperature=.7, max_tokens=1000)
template = """你是一位剧作家。根据戏剧的标题和设定的时代，你的任务是为该标题写一个简介。

标题：{title}
时代：{era}
剧作家：以下是对上述戏剧的简介："""

prompt_template = PromptTemplate(input_variables=["title", "era"], template=template)
# output_key
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="synopsis", verbose=True)

In [17]:
# 这是一个LLMChain，用于根据剧情简介撰写一篇戏剧评论。

template = """你是《纽约时报》的戏剧评论家。根据该剧的剧情简介，你需要撰写一篇关于该剧的评论。

剧情简介：
{synopsis}

来自《纽约时报》戏剧评论家对上述剧目的评价："""

prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="review", verbose=True)

In [18]:
from langchain.chains import SequentialChain

m_overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["synopsis", "review"],
    verbose=True)

In [19]:
m_overall_chain({"title":"三体人不是无法战胜的", "era": "二十一世纪的新中国"})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
你是一位剧作家。根据戏剧的标题和设定的时代，你的任务是为该标题写一个简介。

标题：三体人不是无法战胜的
时代：二十一世纪的新中国
剧作家：以下是对上述戏剧的简介：

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
你是《纽约时报》的戏剧评论家。根据该剧的剧情简介，你需要撰写一篇关于该剧的评论。

剧情简介：


《三体人不是无法战胜的》是一出充满激情和动力的剧情戏剧，发生在二十一个世纪的新中国。它讲述的是一群年轻人在无尽的挑战面前，如何坚持他们的信念，勇敢地面对，最终取得胜利的故事。由于人类文明的挑战，他们展开一场激烈的战斗，抗击外星文明三体的威胁。他们的行动将改变历史进程，并为新中国注入勇气，希望和力量，以实现其未来的理想。

来自《纽约时报》戏剧评论家对上述剧目的评价：

> Finished chain.

> Finished chain.


{'title': '三体人不是无法战胜的',
 'era': '二十一世纪的新中国',
 'synopsis': '\n\n《三体人不是无法战胜的》是一出充满激情和动力的剧情戏剧，发生在二十一个世纪的新中国。它讲述的是一群年轻人在无尽的挑战面前，如何坚持他们的信念，勇敢地面对，最终取得胜利的故事。由于人类文明的挑战，他们展开一场激烈的战斗，抗击外星文明三体的威胁。他们的行动将改变历史进程，并为新中国注入勇气，希望和力量，以实现其未来的理想。',
 'review': '\n\n《三体人不是无法战胜的》是一出非常棒的剧情戏剧，它充满了激情和动力，让观众可以感受到英勇的斗争，热情的友情和坚定的信念。在这个故事中，观众可以看到一群有着不同目标和理念的年轻人，他们不断地努力学习，毅然决然地投身于抗击外星文明三体的斗争。他们的行动和勇气让人们感受到了新中国的力量和希望，实现了他们的未来理想。这部剧的剧情非常引人入胜，它让观众反思自己的信念，并为他们的斗争带来力量和勇气。总之，这部剧值得一看。'}

### Homework

#### 使用 OutputParser 优化 overall_chain 输出格式，区分 synopsis_chain 和 review_chain 的结果